In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/validation-original-aihub/VS_0975.zip
/kaggle/input/validation-original-aihub/VS_1001.zip
/kaggle/input/validation-original-aihub/VS_0929.zip
/kaggle/input/validation-original-aihub/VS_0913.zip
/kaggle/input/validation-original-aihub/VS_0954.zip
/kaggle/input/validation-original-aihub/VS_0995.zip
/kaggle/input/validation-original-aihub/VS_0932.zip
/kaggle/input/validation-original-aihub/VS_0910.zip
/kaggle/input/validation-original-aihub/VS_0981.zip
/kaggle/input/validation-original-aihub/VS_0907.zip
/kaggle/input/validation-original-aihub/VS_0921.zip
/kaggle/input/validation-original-aihub/VS_0920.zip
/kaggle/input/validation-original-aihub/VS_0941.zip
/kaggle/input/validation-original-aihub/VS_0994.zip
/kaggle/input/validation-original-aihub/VS_0926.zip
/kaggle/input/validation-original-aihub/VS_0895.zip
/kaggle/input/validation-original-aihub/VS_0968.zip
/kaggle/input/validation-original-aihub/VS_0965.zip
/kaggle/input/validation-original-aihub/VS_0962.zip
/kaggle/inpu

> # Upload this ipynb to GitHub under your GitHub account won-hj. This can only be undone directly on GitHub.

MIT License
Copyright (c) <year> <copyright holders>

Permission is hereby granted, free of charge, to any person obtaining a copy of this software and associated documentation files (the "Software"), to deal in the Software without restriction, including without limitation the rights to use, copy, modify, merge, publish, distribute, sublicense, and/or sell copies of the Software, and to permit persons to whom the Software is furnished to do so, subject to the following conditions:

The above copyright notice and this permission notice shall be included in all copies or substantial portions of the Software.

THE SOFTWARE IS PROVIDED "AS IS", WITHOUT WARRANTY OF ANY KIND, EXPRESS OR IMPLIED, INCLUDING BUT NOT LIMITED TO THE WARRANTIES OF MERCHANTABILITY, FITNESS FOR A PARTICULAR PURPOSE AND NONINFRINGEMENT. IN NO EVENT SHALL THE AUTHORS OR COPYRIGHT HOLDERS BE LIABLE FOR ANY CLAIM, DAMAGES OR OTHER LIABILITY, WHETHER IN AN ACTION OF CONTRACT, TORT OR OTHERWISE, ARISING FROM, OUT OF OR IN CONNECTION WITH THE SOFTWARE OR THE USE OR OTHER DEALINGS IN THE SOFTWARE.

참고?
https://www.kaggle.com/code/diaaessam/face-verification-and-recognition 
https://www.kaggle.com/code/atulanandjha/face-recognition-starter-trick-code-with-output
https://www.kaggle.com/datasets/atulanandjha/lfwpeople


#import 
# 이미지 로드 및 전처리하기
- 먼저 고급 Keras 사전 처리 유틸리티(예: tf.keras.utils.image_dataset_from_directory) 및 레이어(예: tf.keras.layers.Rescaling)를 사용하여 디스크에서 이미지 디렉터리를 읽습니다.
- 다음으로 tf.data를 사용하여 처음부터 고유한 입력 파이프라인을 작성합니다.
- 마지막으로 TensorFlow Datasets에서 사용할 수 있는 대규모 카탈로그에서 데이터세트를 다운로드합니다.

In [36]:
import keras
import numpy as np
import os
import PIL
import PIL.Image
import tensorflow as tf
import tensorflow_datasets as tfds

In [37]:
import zipfile
import os
import glob

filename = {'train-label':'TL_',
            'train-original':'TS_', 
            'validation-labelling':'VL_', 
            'validation-original':'VS_'}

input_path = '/kaggle/input/'
working_path = '/kaggle/working/'  #문제생기면 여기서 발생 

for k,v in filename.items():
    file_path = input_path + {}-aihub/'.format(k)   
    #.join(glob.glob('.zip'))#.join('-aihub/TL_').join(glob.glob('.zip'))
    print(file_path)
    dir_path = working_path + {}/'.format(k)
    print(dir_path)
    
    
    # 지정된 디렉토리가 없으면 생성합니다.
    if not os.path.exists(dir_path):
        os.makedirs(dir_path)

    #print(os.getcwd())
    # zip 파일 압축 해제
    # 각 압축파일마다 압축해제 
    files = os.listdir(file_path)
    print(files)
    
    for i in range(len(files)):
        with zipfile.ZipFile(file_path+files[i], 'r') as zip_ref:
            zip_ref.extractall(dir_path)

SyntaxError: unterminated string literal (detected at line 14) (3057066489.py, line 14)

In [1]:
#print(os.dirlist('kaggle/working/train-original'))
#!pwd
#!rm *.json
#!ls 
#!ls train-labelling
#!rm -rf train-lebelling
#!ls validation-labelling
#import os
#os.removedirs('/kaggle/working/train-labelling')
#!ls /kaggle/working/

state.db     train-original	   validation-original
train-label  validation-labelling


In [18]:
#데이터셋의 데이터 구조를 출력 - label 
!cat /kaggle/working/train-label/0016_1968_55_00000060_D.json

'''
json 전처리 코드
https://velog.io/@parksh089g/json%ED%98%95%EC%8B%9D-Dataset-%EC%A0%84%EC%B2%98%EB%A6%AC%ED%95%98%EA%B8%B0
0016_1968_55_00000060_D.json -> id_출생년도_나이_
** 크기 조정 후 바운딩 박스나 랜드마크가 변경되는지 확인 **
{
    "filename": "0016_1968_55_00000060_D", #id_birth_age-past
    "id": 16,
    "birth": 1968,
    "age_now": 55,
    "age_past": 55,
    "format": "png",
    "imgsize": "1.324MB",
    "width": 1080,
    "height": 1440,
    "device": "digital",
    "gender": "male",
    "annotation": [
        {
            "box": {
                "x": 3.047921471137073,
                "y": 469.34504908053935,
                "w": 474.59417858244757,
                "h": 519.9905464947254
            },
            "landmark": [
                [
                    311.88763427734375,
                    722.7200927734375
                ],
                [
                    169.94879515695465,
                    559.9612768464077
                ],
                [
                    397.5303955078125,
                    587.8244458053117
                ],
                [
                    165.16746397340108,
                    823.2084790084367
                ],
                [
                    355.03024047511775,
                    823.2270239167203
                ]
            ]
        }
    ]
}

'''

{
    "filename": "0016_1968_55_00000060_D",
    "id": 16,
    "birth": 1968,
    "age_now": 55,
    "age_past": 55,
    "format": "png",
    "imgsize": "1.324MB",
    "width": 1080,
    "height": 1440,
    "device": "digital",
    "gender": "male",
    "annotation": [
        {
            "box": {
                "x": 3.047921471137073,
                "y": 469.34504908053935,
                "w": 474.59417858244757,
                "h": 519.9905464947254
            },
            "landmark": [
                [
                    311.88763427734375,
                    722.7200927734375
                ],
                [
                    169.94879515695465,
                    559.9612768464077
                ],
                [
                    397.5303955078125,
                    587.8244458053117
                ],
                [
                    165.16746397340108,
                    823.2084790084367
                ],
                [
                    

In [35]:
#데이터셋의 데이터 구조를 출력 - label 
!cat /kaggle/working/train-original/0009_1991_14_00000050_F.png
#이미지가 깨져서 나온다 

cat: /kaggle/working/train-original/0009_1991_14_00000050_F.png: No such file or directory


In [9]:
#디렉토리 개수 확인
import os


filename = {'train-label':'TL_',
            'train-original':'TS_', 
            'validation-labelling':'VL_', 
            'validation-original':'VS_'}
dirname = ['train-label', 'train-original', 'validation-labelling', 'validation-original']
len_files = []

ts_path = '/kaggle/working/train-original'#working_path + dir_list[0]
for key in dirname:
    p = '/kaggle/working/'+str(key)
    len_files.append(len(os.listdir(p))) 

print(len_files)
#!tree /kaggle/working/

[0, 2763, 5000, 4000]


In [ ]:
#raw data 확인

import os
import random
import matplotlib.pyplot as plt
from PIL import Image, ImageDraw
from numpy import asarray


In [ ]:
#랜덤 이미지 선택
ts_files = os.listdir('/kaggle/working/train-original')#file_path+file_list[0]
#print('ts-files', ts_files)
# load the photo and extract the face
#랜덤 이미지 대신 모든 이미지에 대해 과정 수행 
train_path = '/kaggle/working/train-original/'
random_file = random.choice(ts_files)
random_path = train_path + random_file
print(random_file)
print(random_path)
#image = Image.open(random_file)


In [ ]:
#훈련 데이터의 아이디에 따라 리스트에 전부 수집
file_id = random_file.split('_')[0]
print(file_id)
i_name = []
count = 0
for i in os.listdir(train_path):
    i_id = i.split('_')[0]
    if i_id == file_id:
        #print('name : ', i)
        i_name.append(i)
        #print('id : ', i_id)
        count = count + 1

print(count)
print(i_name)


In [ ]:
#랜덤 값 출력

#plt.imshow(train_path+random.choice(i_name))
print(train_path+random.choice(i_name))


In [ ]:
import cv2
img1 = cv2.imread(random_path)
img2 = cv2.imread(train_path+random.choice(i_name))
img3 = cv2.imread(train_path+random.choice(i_name))

image1 = Image.open(random_path)
image2 = Image.open(train_path+random.choice(i_name))
image3 = Image.open(train_path+random.choice(i_name))

plt.subplot(1, 3, 1)
plt.imshow(image1)
plt.subplot(1, 3, 2)
plt.imshow(image2)
plt.subplot(1, 3, 3)
plt.imshow(image3)

#ts_files = os.listdir('/kaggle/working/train-original')#file_path+file_list[0]
#print('ts-files', ts_files)
# load the photo and extract the face
#랜덤 이미지 대신 모든 이미지에 대해 과정 수행 
#random_file = '/kaggle/working/train-original/'+ random.choice(ts_files)
#print(random_file)

In [ ]:
#아이디에 따라 폴더 생성 후 파일 분류
print(train_path)

for files in os.listdir(train_path):
    id = files.split('_')[0]
    if not os.path.exists(id):
        os.makedirs(id)
    #if os.path.
        
print(os.listdir('/kaggle/working/0838'))


In [ ]:
#train 서브디렉토리 생성 테스트
print(train_path) #/kaggle/working/train-original/
c = 0
for files in os.listdir(train_path):
    id = files.split('_')[0]
    print('id: ', id)
    if not os.path.exists(train_path+id):
        os.makedirs(train_path+id)
    if os.path.isdir(train_path+id):
        print('true? ', train_path+id)
        
    c += 1
    if c == 10:
        break
#/kaggle/working/0760


In [ ]:
#train 파일 이동
import shutil

c = 0
dir = []
for files in os.listdir(train_path):
    #print(files)
    #if os.path.isdir(train_path+files):
        #dir.append(os.path.realpath(train_path+files))    
    length = len(files)
    if length == 4:
        dir.append(files)
    else:
        shutil.move(train_path+files, train_path+files.split('_')[0])
        #print('id',train_path+files.split('_')[0])
        
        
    #if os.path.isfile(files):
        #print(files)
        #dir.append(files)
    
print(dir) 

In [ ]:
#이동 – validation 파일
validation_path = '/kaggle/working/validation-original/'

import os

def make_dir(path):
    for files in os.listdir(path):
        id = files.split('_')[0]
        if not os.path.exists(path+id):
            os.makedirs(path+id)


    
import shutil

def move_files(path):
    for files in os.listdir(path): 
        length = len(files)
        if length != 4:
            shutil.move(path+files, path+files.split('_')[0])

make_dir(validation_path)
move_files(validation_path)


In [ ]:
#경로삭제
#print(os.path.dirname('/kaggle/working/train-original/0706_2002_21_00000020_D.png'))
#c = 0
for dir in os.listdir('/kaggle/working'):
    print(train_path, dir)
    #print(dir[0])
    if dir[0] == '0':
        print(dir)
        os.rmdir(dir)


In [ ]:
#mtcnn
!pip install mtcnn

In [ ]:
# mtcnn으로 한 가지 이미지에서 얼굴 감지 (예시)
# function for face detection with mtcnn
from PIL import Image, ImageDraw
from numpy import asarray
from mtcnn.mtcnn import MTCNN
import matplotlib.pyplot as plt
import random 

# extract a single face from a given photograph
def extract_face(filename, required_size=(160, 160)):
	# load image from file
	image = Image.open(filename)
	# convert to RGB, if needed
	image = image.convert('RGB')
	# convert to array
	pixels = asarray(image)
    #print(pixes)
    #print('asarray(image)', pixels)
	# create the detector, using default weights
	detector = MTCNN()
	# detect faces in the image
	results = detector.detect_faces(pixels)
    #print('detector.detect_faces(pixels)', results)
	# extract the bounding box from the first face
	x1, y1, width, height = results[0]['box']
    #print('results[0]["box"]', x1, y1, width, height)
	# bug fix
	x1, y1 = abs(x1), abs(y1)
	x2, y2 = x1 + width, y1 + height
	# extract the face
	face = pixels[y1:y2, x1:x2]
	#print('pixels[y1:y2, x1:x2]', face)
    # resize pixels to the model size
	image = Image.fromarray(face)
    #print('Image.fromarray(face)',image)
	image = image.resize(required_size) #리사이즈까지 됐다
    #print('image.resize(required_size)', image)
	face_array = asarray(image)
	return face_array


In [ ]:
#랜덤 이미지 선택
import os 
import random 
ts_files = os.listdir('/kaggle/working/train-original')#file_path+file_list[0]
#print('ts-files', ts_files)
# load the photo and extract the face
#랜덤 이미지 대신 모든 이미지에 대해 과정 수행 
random_id = '/kaggle/working/train-original/'+ random.choice(ts_files) 
random_file = random_id + '/' +random.choice(os.listdir(random_id))
print(random_file)


In [ ]:
#이미지 정렬
import numpy as np
import math
#deepface의 얼굴 회전
def alignment_procedure(img, left_eye, right_eye):
    #this function aligns given face in img based on left and right eye coordinates

    left_eye_x, left_eye_y = left_eye
    right_eye_x, right_eye_y = right_eye

    #-----------------------
    #find rotation direction

    if left_eye_y > right_eye_y:
        point_3rd = (right_eye_x, left_eye_y)
        direction = -1 #rotate same direction to clock
    else:
        point_3rd = (left_eye_x, right_eye_y)
        direction = 1 #rotate inverse direction of clock

    #-----------------------
    #find length of triangle edges

    #a = distance.findEuclideanDistance(np.array(left_eye), np.array(point_3rd))
    #b = distance.findEuclideanDistance(np.array(right_eye), np.array(point_3rd))
    #c = distance.findEuclideanDistance(np.array(right_eye), np.array(left_eye))
    a = np.sqrt(np.sum((np.array(left_eye) - np.array(point_3rd)) ** 2))
    b = np.sqrt(np.sum((np.array(right_eye) - np.array(point_3rd)) ** 2))
    c = np.sqrt(np.sum((np.array(right_eye) - np.array(left_eye)) ** 2))
    print("a: ", a, "b: ", b, "c: ", c) #a:  0.0 b:  53.0 c:  53.0
    #-----------------------

    #apply cosine rule

    if b != 0 and c != 0: #this multiplication causes division by zero in cos_a calculation

            cos_a = (b*b + c*c - a*a)/(2*b*c)
        angle = np.arccos(cos_a) #angle in radian
        angle = (angle * 180) / math.pi #radian to degree

        #-----------------------
        #rotate base image

        if direction == -1:
            angle = 90 - angle

        img = Image.fromarray(img)
        img = np.array(img.rotate(direction * angle))

    #-----------------------
    plt.imshow(img)
    return img #return img anyway


In [ ]:
#함수아닌 mtcnn 사용 위한 부분

# function for face detection with mtcnn
from PIL import Image, ImageDraw
from numpy import asarray
from mtcnn.mtcnn import MTCNN
import matplotlib.pyplot as plt
import random

#첫 번째 단계는 PIL 라이브러리와 open() 함수 를 사용하여 이미지를 NumPy 배열로 로드
#이미지에 알파 채널이 있거나 흑백인 경우를 대비해 이미지를 RGB로 변환

# load image from file

#filename -> random_file 사용
image = Image.open(random_file)
raw_image = image
# convert to RGB, if needed
image = image.convert('RGB')
# convert to array
pixels = asarray(image)

#다음으로 MTCNN 얼굴 감지기 클래스를 만들고 이를 사용하여 로드된 사진에서 모든 얼굴을 감지
# create the detector, using default weights
detector = MTCNN()
# detect faces in the image
results = detector.detect_faces(pixels)

#결과는 경계 상자 목록

#음수 픽셀 인덱스를 반환 시 절대값 사용
# extract the bounding box from the first face
x1, y1, width, height = results[0]['box']
# bug fix
x1, y1 = abs(x1), abs(y1)
x2, y2 = x1 + width, y1 + height

#키포인트
keypoints = results[0]['keypoints']
left_eye = keypoints["left_eye"]
right_eye = keypoints["right_eye"]
print(left_eye, right_eye)
pixels = alignment_procedure(pixels,left_eye ,right_eye)
if(pixels is not None):
    print("pixels is not none")
#draw = ImageDraw.Draw(image)
#draw.rectangle(((x1,y1), (x2, y2)), fill="blue", outline=(255,0,0), width=3)
#draw.ellipse([(left_eye[0]-2, left_eye[1]+2, left_eye[0]+2, left_eye[1]-2)], fill="blue")
#draw.rectangle(((x1,y1-height//10),(x1+width//2, y1)), fill=(255,0,0))
#image.save(ryan_path.split(os.sep)[-1].split('.')[0]+'random_test'+'.jpg')
#image.show()
# extract the face
face = pixels[y1:y2, x1:x2]

#PIL 라이브러리를 사용하여 이 작은 얼굴 이미지의 크기를 필요한 크기로 조정
#160x160
# resize pixels to the model size
image = Image.fromarray(face)
image = image.resize((160, 160))
face_array = asarray(image)

plt.imshow(face_array)


In [ ]:
#결과 비교
#print(pixels)
#얼굴 추출 결과 비교 
import matplotlib.pyplot as plt
import PIL.Image as pil

#img = pil.open(random_file)
#img
#plt.sub
#plt.imshow(random_file)
#print(random_file)

plt.subplot(1, 3, 1)
plt.imshow(raw_image)
plt.subplot(1, 3, 2)
plt.imshow(pixels)
plt.subplot(1, 3, 3)
plt.imshow(face_array)


# 임베딩 

In [ ]:
#임베딩 사전 준비 (시도)
import glob

className = glob(train_path + "/*")
NumberofClass = len(className)
print("NumberofClass:", NumberofClass)


In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import tensorflow as tf
import keras
from keras.models import Sequential
from keras.preprocessing.image import  load_img, img_to_array #ImageDataGenerator,
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from keras.layers import Dense, Flatten, BatchNormalization, Dropout
from keras.applications import InceptionV3
from glob import glob


In [ ]:
train_datagen = ImageDataGenerator(rescale=1/255)
validation_datagen=ImageDataGenerator(rescale=1/255)
test_datagen=ImageDataGenerator(rescale=1/255)


In [ ]:
train_path = '/kaggle/working/train-original/'
validation_path = '/kaggle/working/validation-original/'

batch_size = 256
train_datagen = ImageDataGenerator(rescale=1/255,
                  shear_range=0.3,
                  horizontal_flip=True,
                  zoom_range=0.3
                  )
val_datagen = ImageDataGenerator(rescale=1/255)

train_generator = train_datagen.flow_from_directory(
                train_path,
                target_size=(224,224),
                batch_size=batch_size,
                color_mode="rgb",
                class_mode="categorical"
                )

val_generator = val_datagen.flow_from_directory(
                validation_path,
                target_size=(224,224),
                batch_size=batch_size,
                color_mode="rgb",
                class_mode="categorical"
                )



# 라이언 테스트 

In [ ]:
#라이언으로 테스트
#라이언 추출 - 얼굴 부분만
from pathlib import Path
import os
#for files in Path('ryan-example').glob("*/*"):
    #name = files.parent.name
    #images = extract_face(name)
    #plt.imshow(images)
ryan_path = "/kaggle/input/ryan-example"
#/kaggle/input/ryan-example/KakaoTalk_20240604_025055676.jpg
print(ryan_path) #/kaggle/input/ryan-example/
print(os.listdir("/kaggle/input/ryan-example")) #['ryan2.jpg', 'ryan1.jpg']

#fix, ax = plt.subplot(1, 2, figsize=(8,3))
img_list = []
fig = plt.figure()
i = 1
for files in os.listdir(ryan_path):
    print(files)
    pixel = extract_face(ryan_path+'/'+files, required_size=(160, 160))
    ax = fig.add_subplot(1, 2, i)
    ax.imshow(pixel)
    i += 1

plt.show()

##ryan2는 정면을 바라보는 라이언의 얼굴이어야하는데 아래에 나오는 논문의 얼굴 중 하나가 추출되어 이상한 사진이 나옴
##대신 휴대폰으로 얼굴 사진을 찍는 것을 생각하면 그 정도까지의 복잡함은 고려 안해도 된다고 생각함


In [ ]:
#회전하는 함수 대체

In [ ]:
image = Image.open(ryan_path+'/ryan1.jpg')
image = image.convert('RGB')
pixels = asarray(image)

detector = MTCNN()

results = detector.detect_faces(pixels)
#print('detect_face:', results)
x1, y1, width, height = results[0]['box']
x1, y1 = abs(x1), abs(y1)
x2, y2 = x1 + width, y1 + height

face = pixels[y1:y2, x1:x2] 
keypoints = results[0]['keypoints']
left_eye = keypoints["left_eye"]
right_eye = keypoints["right_eye"]

le_x, le_y = left_eye
re_x, re_y = right_eye

print("eyes: ", left_eye, 
      le_x, le_y, 
      right_eye, 
      re_x, re_y)

#face = alignment_procedure(face, left_eye, right_eye)
#plt.imshow(face)



image = Image.fromarray(face)
image = image.resize((160, 160))
ryan_img = 'ryan2.jpg'
face_array = asarray(image)




plt.imshow(face_array)

#plt.imshow(ryan_img)

#import cv2
#face.astype(np.uint8)
#img = cv2.circle(face_array, left_eye,2, (0,0,0), 1)
#img = cv2.circle(ryan_img, left_eye, 3, (0,0,0),)
#cv2.imshow('',img)

'''fig = plt.figure()
i = 1
for files in os.listdir(ryan_path):
    print(files)
    pixel = extract_face(ryan_path+'/'+files, required_size=(160, 160))
    ax = fig.add_subplot(1, 2, i)
    ax.imshow(pixel)
    i += 1

plt.show()'''
#정렬실패


In [ ]:
from PIL import ImageDraw
import cv2

ryan = cv2.imread('/kaggle/input/ryan-example/ryan1.jpg')

cv2.rectangle(ryan, (x1, y1), (x2, y2), (0,155,255),2)
cv2.circle(ryan,(keypoints['left_eye']), 2, (0,155,255), 2)
cv2.circle(ryan,(keypoints['right_eye']), 2, (0,155,255), 2)

#cv2.imwrite("ryan1_boxpoints.jpg", ryan)
#cv2.namedWindow("ryan1_boxpoints")
#cv2.imshow("ryan1_boxpoints",ryan)


In [ ]:
from mtcnn import MTCNN
import cv2

detector = MTCNN()

img = cv2.imread("img.jpg")
detections = detector.detect_faces(img)

for detection in detections:
    score = detection["confidence"]
    if score > 0.90:
        x, y, w, h = detection["box"]
        detected_face = img[int(y):int(y+h), int(x):int(x+w)]
        
        
#출처: https://a292run.tistory.com/entry/Deep-Face-Detection-with-MTCNN-in-Python-1 [Dead & Street:티스토리]
#정확도에 따른 분류 
